In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from sklearn.metrics import accuracy_score
import os
import gc
import random

In [8]:
%%time
#Для начала создадим объект numpy.ndarray из директории с исходными изображениями
#Для корректной работы в директории с данным ноутбуком должна находиться разархивированная папка fruits-360
#Данная процедура использует функции библиотеки OpenCV для чтения изображений, изменения размера
#и создания массива из лейблов
training_fruit_img = []
training_label = []
for dir_path in glob.glob("fruits-360/Training/*"):
    img_label = dir_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(dir_path, "*.jpg")):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        training_fruit_img.append(img)
        training_label.append(img_label)
training_fruit_img = np.array(training_fruit_img)
training_label = np.array(training_label)
len(np.unique(training_label))

Wall time: 1min 57s


120

In [9]:
%%time
#Повторим для тестовых изображений
test_fruit_img = []
test_label = []
for dir_path in glob.glob("fruits-360/Test/*"):
    img_label = dir_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(dir_path, "*.jpg")):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        test_fruit_img.append(img)
        test_label.append(img_label)
test_fruit_img = np.array(test_fruit_img)
test_label = np.array(test_label)
len(np.unique(test_label))

Wall time: 38.8 s


120

In [10]:
#Самописные лейбл-энкодер и декодер
label_to_id = {v : k for k, v in enumerate(np.unique(training_label))}
id_to_label = {v : k for k, v in label_to_id.items()}

In [11]:
#Кодируем классы для трейна и теста
training_label_id = np.array([label_to_id[i] for i in training_label])
test_label_id = np.array([label_to_id[i] for i in test_label])

In [12]:
#Нормализуем числовые значения изображений, чтобы их область значения была в интервале [0, 1]
training_fruit_img = training_fruit_img / 255.0

In [13]:
test_fruit_img = test_fruit_img / 255.0

In [14]:
#Построим исходную сеть из 4 свёрточных слоёв и двух полносвязных
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape = (64, 64, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(120, activation = "softmax"))

In [15]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'Adam', metrics = ['accuracy'])

In [17]:
%%time
model.fit(training_fruit_img, training_label_id, batch_size = 256, epochs = 10)

Epoch 1/10
60498/60498 [==============================] - 59s 968us/step - loss: 1.1735 - accuracy: 0.7099
Epoch 2/10
60498/60498 [==============================] - 56s 928us/step - loss: 0.0926 - accuracy: 0.9724
Epoch 3/10
60498/60498 [==============================] - 57s 942us/step - loss: 0.0400 - accuracy: 0.9884
Epoch 4/10
60498/60498 [==============================] - 56s 932us/step - loss: 0.0261 - accuracy: 0.9924
Epoch 5/10
60498/60498 [==============================] - 56s 927us/step - loss: 0.0977 - accuracy: 0.9762
Epoch 6/10
60498/60498 [==============================] - 56s 930us/step - loss: 0.0042 - accuracy: 0.9992
Epoch 7/10
60498/60498 [==============================] - 56s 924us/step - loss: 0.1128 - accuracy: 0.9739
Epoch 8/10
60498/60498 [==============================] - 56s 921us/step - loss: 0.0069 - accuracy: 0.9987
Epoch 9/10
60498/60498 [==============================] - 56s 929us/step - loss: 9.5930e-04 - accuracy: 1.0000
Epoch 10/10
60498/60498 [========

In [18]:
%%time
predicted_classes = model.predict_classes(test_fruit_img)

Wall time: 7.73 s


In [19]:
accuracy_score(predicted_classes, test_label_id)

0.973183978275628

In [25]:
#Сохраним полученные тестовые и тренировочные данные в файл для более удобного дальнейшего использования
#В моём случае препроцессинг занимает слишком много оперативной памяти и многократное повторение может вызвать ошибки
np.save('training_fruit_img.npy', training_fruit_img)
np.save('test_fruit_img.npy', test_fruit_img)

In [9]:
np.save('training_label_id.npy', training_label_id)
np.save('test_label_id.npy', test_label_id)